# Imports

In [ ]:
import gzip
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import shutil

from copy import deepcopy

# Global Variables

In [ ]:
drive_dir = "/content/drive/MyDrive/files/"
data_dir = "/content/drive/MyDrive/Group Project/Dataset/"
prep_dir = data_dir + "Preprocessed/"
multi_dir = prep_dir + "Multi Cell/data/"
single_dir = prep_dir + "Single Cell/"
inter_dir = single_dir + "intermediate_GSE132465_GEO_processed_CRC_10X_natural_log_TPM_matrix/"

# Make Directories

In [ ]:
os.makedirs("num_files/")
os.makedirs("sample_files/")
os.makedirs("batch_files/")
os.makedirs("preprocessed/")
os.makedirs("preprocessed_batches/")

FileExistsError: ignored

# Break File into Smaller Files

In [ ]:
f_name = "GSE132465_GEO_processed_CRC_10X_natural_log_TPM_matrix.txt"
with open(single_dir + f_name) as file:
    lines = [[]]

    c = 0
    cols = None
    for l in file.readlines():
        lines.append(l)
        if cols is None:
            cols = lines[-1]
            lines = [[]]
        c += 1

        amt = 500
        if c % amt == 0:
            lines[0] = cols
            with open("num_files/%d.txt"%(c / amt), "w") as file:
                file.writelines(lines)
                del lines
                lines = [[]]
            # if c / amt == 2:
            #     break

    c = int(c / amt) + 1
    lines[0] = cols
    with open("num_files/%d.txt"%c, "w") as file:
        file.writelines(lines)
        del lines

# Assemble Samples

In [ ]:
c = 1
mode = None
while True:
    f_name = "num_files/%d.txt"%c
    print(c)
    if os.path.isfile(f_name):
        # if c > 3:
        #     break
            
        df = pd.read_csv(f_name, sep="\t") #, index_col="Index")
        df = df.transpose()
        
        for i, row in df.iterrows():
            
            f_name = "sample_files/%s.txt"%row.name
            txt = ""
            if os.path.exists(f_name):
                mode = "a"
                txt += ","
            else:
                mode = "w"
            with open("sample_files/%s.txt"%row.name, mode) as file:

                for v in row.values:
                    txt += str(v) + ","
                # print(txt)
                file.write(txt[:-1])
        
        del df
    else:
        break
        
    c += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


# Combine Samples into Batches of 10

In [ ]:
cols = None
with open("sample_files/Index.txt", "r") as file:
    cols = file.read()
    cols = cols.strip()
    cols = cols.split(",")
    cols = ["sample"] + cols

In [ ]:
batch = []
batch_c = 0
c = 0
for f in os.listdir("sample_files/"):
    if "SMC" in f and ".txt" in f:
        if c % 1000 == 0:
            print(c)

        with open("sample_files/" + f, "r") as file:
            txt = file.read()
            txt = txt.strip()

            txt = [f.split(".")[0]] + [float(x) for x in txt.split(",")]
            batch.append(txt)

        if len(batch) == 10:
            df = pd.DataFrame(batch, columns=cols)
            df.to_csv("batch_files/batch_%d.csv"%batch_c, index=False)
            batch = []
            batch_c += 1

        c += 1

df = pd.DataFrame(batch, columns=cols)
df.to_csv("batch_files/batch_%d.csv"%batch_c, index=False)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000


# Preprocess Samples

In [ ]:
def handle_file(f):

    # Read file
    df = pd.read_csv("batch_files/" + f, index_col="sample")

    # Get sequences
    seqs = []
    for i, row in df.iterrows():
        n = row.name
        vals = [x for x in row.values if x > 0]

        while len(vals) < 8000:
            vals = vals + [0.0] + vals

        while len(vals) > 200:
            seqs.append(vals[:200] + [n])
            vals = vals[200:]

    # To .csv file
    df = pd.DataFrame(seqs)
    cols = list(df.columns)
    cols[-1] = "label"
    df.columns = cols
    df.to_csv(inter_dir + f, index=False)


# Iterate over files
c = 0
for f in os.listdir("batch_files/"):
    if c % 1000 == 0:
        print(c)
    handle_file(f) 
    c += 1

0
1000
2000
3000


KeyboardInterrupt: ignored